Практика
Предсказываем
клики в рекламе

Постановка задачи
- 40 млн записей о сессиях пользователей в Mobile Web
- история их взаимодействия с рекламными баннерами за 10
дней
- Задача:
- мы считаем, что пользователь уже увидел баннер
- для данной сессии нужно предсказать кликнет ли пользователь
на баннер или нет

In [1]:
import pandas as pd
import numpy as np


In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

Dataset:
- train: 40 млн записей за 10 дней истории пользователей
- test: 4.5 млн записей за следующий 11й день
- Для нашего кейса семплируем 5 млн данных из обучающей
выборки

In [3]:
df = pd.read_csv('./data/raw/sampled_train_50k.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data/raw/sampled_train_50k.csv'

Признаки
- click - 0/ 1 таргет
- hour - время и дата показа рекламы
- banner_pos - расположение баннера
- site*, app* - информация сайте/ приложении издателя, где отображается баннер
- device* - информация о девайсе пользователя
- C1, C14-C21 - зашифрованные признаки (предположительно геолокация,
часовой пояс и формат баннера)

- Смотрим долю кликов (Click Through Rate, CTR): 16.9%

In [ ]:
df.head()

,Unnamed: 0,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1148151,13614095229350157618,1,14102222,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,a99f214a,7ed30f6c,8a4875bd,1,0,20634,320,50,2374,3,39,-1,23
1,10990,5642023972583688480,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,a07251cf,711ee120,1,0,15703,320,50,1722,0,35,-1,79
2,2085412,8696223902141438774,0,14102502,1002,0,6b175d24,a1b0ebd0,50e219e0,ecad2386,7801e8d9,07d7df22,630e9b36,664e14a5,766b28b3,0,0,20108,320,50,2299,2,1327,100084,52
3,3241440,16692284365181240212,0,14102717,1005,0,85f751fd,c4e18dd6,50e219e0,92f5800b,ae637522,0f2161f8,a99f214a,9d834082,0bcabeaf,1,3,21189,320,50,2424,1,161,100193,71
4,3360862,10524265036687563821,0,14102801,1005,0,85f751fd,c4e18dd6,50e219e0,4b337c54,d9b5648e,0f2161f8,a99f214a,e7e6dbed,9b5ce758,1,0,21611,320,50,2480,3,297,100111,61


In [ ]:
df['click'].value_counts(normalize=True)

click
0    0.82948
1    0.17052
Name: proportion, dtype: float64

.EDA: device_id
- Уникальных device_id 600k
- 82% всего трафика приходится на один девайс

In [11]:
df.describe()

,Unnamed: 0,id,click,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,5.000000e+04,5.000000e+04,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.00000,50000.000000,50000.000000
mean,2.479551e+06,9.210294e+18,0.170520,1.410255e+07,1004.967220,0.283640,1.016000,0.329920,18821.575300,318.852080,60.415560,2109.89090,1.437140,224.34050,52626.624280,83.054720
std,1.439601e+06,5.322361e+18,0.376093,2.959279e+02,1.094591,0.507221,0.529669,0.855481,4941.858313,21.420766,48.418154,607.81724,1.327887,349.52954,49991.160513,69.979535
min,5.300000e+01,1.173645e+15,0.000000,1.410210e+07,1001.000000,0.000000,0.000000,0.000000,375.000000,120.000000,20.000000,112.00000,0.000000,33.00000,-1.000000,1.000000
25%,1.233248e+06,4.604551e+18,0.000000,1.410230e+07,1005.000000,0.000000,1.000000,0.000000,16859.000000,320.000000,50.000000,1823.00000,0.000000,35.00000,-1.000000,23.000000
50%,2.462724e+06,9.194584e+18,0.000000,1.410252e+07,1005.000000,0.000000,1.000000,0.000000,20345.000000,320.000000,50.000000,2316.00000,2.000000,39.00000,100031.000000,61.000000
75%,3.724000e+06,1.384460e+19,0.000000,1.410281e+07,1005.000000,1.000000,1.000000,0.000000,21883.000000,320.000000,50.000000,2524.25000,3.000000,171.00000,100084.000000,100.000000
max,4.999892e+06,1.844647e+19,1.000000,1.410302e+07,1012.000000,7.000000,5.000000,5.000000,24041.000000,1024.000000,1024.000000,2756.00000,3.000000,1839.00000,100248.000000,255.000000
